In [21]:
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import metrics, Input
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [17]:
METRICS = [metrics.RootMeanSquaredError(name='rms'), metrics.MeanAbsoluteError(name='mae')]

In [26]:
ENCODING_DIM = 16 #Desired Dimension
BATCH_SIZE = 64
EPOCHS = 2

In [27]:
def make_and_train_autoencoder(X_train, metrics=METRICS):
    
    len_input_output = X_train.shape[-1]
    input_ = Input(shape=(len_input_output,))
    encoded = Dense(units=ENCODING_DIM*2, activation="relu")(input_)
    bottleneck = Dense(units=ENCODING_DIM, 
                       activation="relu")(encoded)
    decoded = Dense(units=ENCODING_DIM*2, 
                    activation="relu")(bottleneck)
    output = Dense(units=len_input_output, 
                    activation="linear")(decoded)
    #Training is performed on the entire autoencoder
    autoencoder = Model(inputs=input_, outputs=output)
    autoencoder.compile(optimizer='adam', loss='mean_squared_error',
                        metrics=[metrics])
    autoencoder.fit(X_train, X_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS)
    #Use only the encoder part for dimensionality reduction
    encoder = Model(inputs=input_, outputs=bottleneck)
    return autoencoder, encoder

In [32]:
df = pd.read_csv('training_ROS.csv')
qw = pd.read_csv('validation_set.csv')

In [29]:
X, y = df.iloc[:,:-1], df.iloc[:, -1]

In [30]:
auto, encode = make_and_train_autoencoder(X, metrics=METRICS)

Epoch 1/2
329/329 [==============================] - 5s 7ms/step - loss: 0.0106 - rms: 0.0865 - mae: 0.0451
Epoch 2/2
329/329 [==============================] - 2s 7ms/step - loss: 0.0015 - rms: 0.0387 - mae: 0.0234


In [42]:
q = encode.predict(X)

In [43]:
q.shape

(21000, 16)

In [44]:
from sklearn.ensemble import AdaBoostClassifier 

In [46]:
#oversampling ada 

ada = AdaBoostClassifier(n_estimators=100,learning_rate=1)
ada_oversamp=ada.fit(x_ov,y_ov)
ada_oversamp_prediction=ada_oversamp.predict(x_test)
ada_oversamp_accuracy=accuracy_score(ada_oversamp_prediction,y_test)
ada_oversamp_f1=f1_score(ada_oversamp_prediction,y_test)
ada_oversamp=[ada_oversamp_accuracy,ada_oversamp_f1]
accuracy_over_sampling.append(ada_oversamp_accuracy)
f1_over_sampling.append(ada_oversamp_f1)

NameError: name 'x_ov' is not defined

In [ ]:
def optimal_epoch(max_val, X_train, metrics=METRICS):
    auto = []
    encode = []
    for i in range(max_val):
        EPOCHS = i
        autoencoder, encoder = make_and_train_autoencoder(X_train, metrics=METRICS)
        auto.append(autoencoder)
        encode.append(encoder)
    return auto, encode
        